# Exemplo de Conexão Spark com MinIO

Este notebook demonstra como conectar o Spark ao MinIO e realizar operações básicas de leitura/escrita de dados.

## 1. Criar SparkSession com configuração para MinIO

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Criar SparkSession com configurações para MinIO
spark = SparkSession.builder \
    .appName("MinIO-Spark-Example") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
print(f"Spark UI: http://localhost:4040")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/28 17:20:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Version: 3.5.7
Spark UI: http://localhost:4040


## 2. Criar dados de exemplo

In [ ]:
# Criar um DataFrame de exemplo
data = [
    (1, "João", 25, "Engenharia"),
    (2, "Maria", 30, "Medicina"),
    (3, "Pedro", 22, "Direito"),
    (4, "Ana", 28, "Administração"),
    (5, "Carlos", 35, "TI")
]

columns = ["id", "nome", "idade", "curso"]
df = spark.createDataFrame(data, columns)

df.show()
df.printSchema()

## 3. Salvar dados no MinIO

**Nota:** Antes de executar esta célula, certifique-se de criar o bucket 'dados' no MinIO Console (http://localhost:9001)

In [ ]:
# Salvar como CSV
df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("s3a://dados/alunos/csv/")

print("✓ Dados salvos como CSV no MinIO")

In [ ]:
# Salvar como Parquet
df.write \
    .mode("overwrite") \
    .parquet("s3a://dados/alunos/parquet/")

print("✓ Dados salvos como Parquet no MinIO")

## 4. Ler dados do MinIO

In [ ]:
# Ler CSV do MinIO
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("s3a://dados/alunos/csv/")

print("Dados lidos do CSV:")
df_csv.show()

In [ ]:
# Ler Parquet do MinIO
df_parquet = spark.read.parquet("s3a://dados/alunos/parquet/")

print("Dados lidos do Parquet:")
df_parquet.show()

## 5. Análises básicas

In [ ]:
# Estatísticas descritivas
df_parquet.describe().show()

In [ ]:
# Contar por curso
df_parquet.groupBy("curso").count().show()

In [ ]:
# Idade média
df_parquet.agg(avg("idade").alias("idade_media")).show()

## 6. Usar Delta Lake (opcional)

In [ ]:
# Salvar como Delta Lake
df.write \
    .format("delta") \
    .mode("overwrite") \
    .save("s3a://dados/alunos/delta/")

print("✓ Dados salvos em formato Delta Lake")

In [ ]:
# Ler Delta Lake
df_delta = spark.read.format("delta").load("s3a://dados/alunos/delta/")
df_delta.show()

## 7. Visualização com Pandas e Matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Converter para Pandas
pdf = df_parquet.toPandas()

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Criar gráfico
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras - Contagem por curso
curso_counts = pdf['curso'].value_counts()
curso_counts.plot(kind='bar', ax=ax1, color='steelblue')
ax1.set_title('Quantidade de Alunos por Curso', fontsize=14, fontweight='bold')
ax1.set_xlabel('Curso')
ax1.set_ylabel('Quantidade')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de distribuição de idade
pdf['idade'].plot(kind='hist', bins=10, ax=ax2, color='coral', edgecolor='black')
ax2.set_title('Distribuição de Idade', fontsize=14, fontweight='bold')
ax2.set_xlabel('Idade')
ax2.set_ylabel('Frequência')

plt.tight_layout()
plt.show()

print(f"\nEstatísticas:\n{pdf.describe()}")

## 8. Finalizar sessão

In [ ]:
# Parar SparkSession
# spark.stop()
# print("✓ Spark Session finalizada")